In [1]:
import qforte as qf
from qforte import *

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
$$

# Gate, Circuits, and Operators

This notebook is a tutorial for the use of gates, circuits, and operators in QForte. 
It will therefore focus on usage of the three corresponding classes `QuantumGate`, `QuantumCircuit`, and `QuantumOperator`.
All three classes are implemented in C++ but are exposed in Python for ease of use.
It will also discuss the usage of second quantized operators (via the `SQOperator` class) and how it relates to the other classes.

## Quantum gates



QForte, like many packages related to quantum simulation (or more generally any subfield of quantum computing) uses a class that represents elementary quantum gates.
The `QuantumGate` is the most fundamental building block for all quantum algorithms in QForte.
Some of the most pertinent to quantum simulation are the Pauli gates ($\hat{Z}$, $\hat{Y}$, and $\hat{Z}$), the Hadamard gate $\hat{H}$, the controlled NOT [CNOT] gate, and the parametric z rotation gate $\hat{R}_z(\theta)$. A full list of gates can be found in the QForte documentation.
Note that all quantum gates represent *unitary* operations.

The `QuantumGate` class has several important attributes including a string (`label_`) which gives its identity, the integer `target_` and `control_` qubit indices, and the matrix of complex values `gate_`.
Instantiating a `QuantumGate` is simple, and is done via the `make_gate()` member function.

Consider the following examples:
> instantiate the Pauli $\hat{X}$ gate that will target the qubit $q_4$ and print its representation using the `str()` function.

In [2]:
# Specify target and controll qubits
target_idx = 4
control_idx = 4

X_4gate = qf.make_gate('X', target_idx, control_idx)

print(X_4gate.str())

X4


**NOTE:** for single qubit gates (such as the Pauli gates), one passes the same value for the target and control index.

> Instantiate the CNOT gate that will target the qubit $q_4$ and use the qubit $q_1$ as a control.

In [3]:
# Specify target and controll qubits
target_idx = 4
control_idx = 1

CNOT_4_1gate = qf.make_gate('CNOT', target_idx, control_idx)

print(CNOT_4_1gate.str())

CNOT4_1


## Quantum circuits

Mathematically speaking, a quantum circuit, commonly referred to as a unitary ($\hat{U}$), is represented by a product of quantum gates, making the overall circuit itself a unitary operation.
The `QuantumCircuit` class operates at one level above the `QuantumGate` class. The primary attribute is the vector `gates_` of `QuantumGate` objects. 


For example:

> One can instantiate an empty `QuantumCircuit` without the use of a maker function.

In [4]:
circ = qf.QuantumCircuit()

> Add a gate via the `add_gate()` member function.

In [5]:
circ.add_gate(X_4gate)
circ.add_gate(CNOT_4_1gate)

print(circ.str())

[X4 CNOT4_1]


## Circuits for exponenntial operators.

Although any product of elementary gates technically constitutes a circuit, one of the most important circuit structures in quantum simulation is that which represents unitiaries of the form
\begin{equation}
\label{eq:exp_pt}
e^{i \theta_\ell \hat{P}_\ell} 
%= e^{\theta_\ell \prod_k^{n_\ell}  \hat{V}^{(\ell) \dagger}_k \hat{Z}^{(\ell)}_k \hat{V}^{(\ell)}_k}
= \Bigg(\prod_k^{n_\ell} \hat{V}_k^{(\ell)} \Bigg)^\dagger \Bigg(\prod_k^{n_\ell-1} c\hat{X}_{k, k+1}^{(\ell)} \Bigg)^\dagger \hat{R}_z(2 \theta_\ell) \Bigg(\prod_k^{n_\ell-1} c\hat{X}_{k, k+1}^{(\ell)} \Bigg) \Bigg(\prod_k^{n_\ell} \hat{V}_k^{(\ell)} \Bigg),
\end{equation} 
where
\begin{equation}
\hat{P}_\ell = \prod_k^{n_\ell} \hat{\sigma}^{(\ell)}_k
\end{equation}
is a unique product of $n_\ell$ Pauli operators  ($\hat{X}$, $\hat{Y}$, or $\hat{Z}$). 
In this case, $k=(p, [X, Y,$ or $ Z])$ is a compound index over the products in a term $\hat{P}_\ell$ and denotes the qubit ($p$) and specific Pauli gate.
The transformation unitary $\hat{V}^{(\ell)}_k$ is a one qubit gate that transforms $\hat{X}$ or $\hat{Y}$ into $\hat{Z}$.


In QForte this requires one to pass a coefficient and q `QuantumCircuit` to the utility function `exponentiate_single_term().`

> Build the circuit corresponding to $\exp(-i 0.5 \hat{X}_3 \hat{Z}_2 \hat{Z}_1 \hat{Z}_0)$ 

In [6]:
# Construct the desired preliminary circuit (X3 Z2 Z1 Z0)
circ = qf.QuantumCircuit()
circ.add_gate(qf.make_gate('Z', 0, 0))
circ.add_gate(qf.make_gate('Z', 1, 1))
circ.add_gate(qf.make_gate('Z', 2, 2))
circ.add_gate(qf.make_gate('X', 3, 3))
print('\n The origional unitary circuit \n',circ.str())

# Define the factor (-i theta)
theta = 0.5
factor = -1.0j * theta

# Construct the unitary for the exonential
Uexp, phase = exponentiate_single_term(factor, circ)
print('\n The exponential unitary circuit \n',Uexp.str())


 The origional unitary circuit 
 [Z0 Z1 Z2 X3]

 The exponential unitary circuit 
 [H3 cX1_0 cX2_1 cX3_2 Rz3 cX3_2 cX2_1 cX1_0 H3]


## Quantum operators

The outer-most operations class in QForte is the `QuantumOperator` class. Again, mathematically speaking, a generic quantum operator $\hat{O}$ is given by a linear combination of $N_\ell$ unitiaries ($\hat{U}_\ell$) as
\begin{equation}
\hat{O} = \sum_\ell u_\ell \hat{U}_\ell,
\end{equation}
where $u_\ell$ is a complex coefficient.
It is important to note that applying a `QuantumOperator` to a quantum state is in general **not a physically valid** operation.

The key attribute of the`QuantumOperator` class is `terms_`: a vector of pairs of the form `<complex::double, QuantumCircuit>`.
Importantly the `QuanntumOperator` class is used to represent important objects such as the Hamiltonian $\hat{\mathcal{H}}$ or the cluster operator $\hat{T}$ in QForte's algorithmic implementations.

> An empty `QuantumOperator` can  likewise be instantiated without a maker function 

In [7]:
q_op = qf.QuantumOperator()

> And can be appended with a new coefficient and circuit via `add_term()` or with an existing operator via `add_op()`. 

In [8]:
u1 = 0.5
u2 = 0.5j

q_op.add_term(u1, circ)
q_op.add_term(u2, circ)

print(q_op.str())

+0.500000[0 1 2 3]
+0.500000j[0 1 2 3]


## Second quantized operators

QForte also supports operators in the form of second quantization, that is, operators comprised of fermionic annihilation ($\hat{a}_p$) and creation ($\hat{a}_p^\dagger$) operators. 
The `SQOperator` class functions very similarly to the `QuantumOperator` class, but utilizes slightly different syntax.

**NOTE:** Second quantized operators in QForte always assume (i) that particle number is conserved (i.e. each term of the quantum operator must have an even number of annihilators and creators), and (ii) that the individual fermionic operators are normal ordered within a term.

In [9]:
sq_op = qf.SQOperator()

In [10]:
h1 = 0.5 
a1 = [1,2]

h2 = -0.25j
a2 = [4,2,3,1]

sq_op.add_term(h1, a1) 
sq_op.add_term(h2, a2) 

print(sq_op.str())

 +0.500000 ( 1^ 2 )
 -0.250000j ( 4^ 2^ 3 1 )



The second quantized operators can then be transformed to the quantum operator representation (given as a linear combination of products of Pauli operators) via the Jordan-Wigner transformation.
Under this transformation, there is a one-to-one mapping between a spin orbital $\phi_p$ and qubit $q_p$ such that the fermionic annihilation ($\hat{a}_{p}$) and creation ($\hat{a}^{\dagger}_{p}$) operators are represented by
\begin{equation}
\hat{a}_{p} = \frac{1}{2} \Big( \hat{X}_p + i \hat{Y}_p \Big) \hat{Z}_{p-1} \dots \hat{Z}_0, 
\end{equation}
and,
\begin{equation}
\hat{a}^{\dagger}_{p} = \frac{1}{2} \Big( \hat{X}_p - i \hat{Y}_p \Big) \hat{Z}_{p-1} \dots \hat{Z}_0. 
\end{equation}

> Print the operator defined above as qubit operators after applying the Jordan-Wigner transformation.

In [11]:
pauli_op = sq_op.jw_transform()
print(pauli_op.str())

+0.015625j[X1 X2 Y3 Y4]
-0.015625[Y1 X2 Y3 Y4]
+0.015625j[Y1 Y2 X3 X4]
+0.015625j[X1 X2 X3 X4]
+0.015625[X1 Y2 Y3 Y4]
-0.015625[Y1 Y2 Y3 X4]
+0.015625j[Y1 Y2 Y3 Y4]
+0.015625j[Y1 X2 X3 Y4]
-0.015625[Y1 X2 X3 X4]
+0.015625[X1 Y2 X3 X4]
+0.015625[Y1 Y2 X3 Y4]
-0.015625j[Y1 X2 Y3 X4]
-0.015625j[X1 Y2 X3 Y4]
+0.015625j[X1 Y2 Y3 X4]
+0.125000[X1 X2]
-0.015625[X1 X2 Y3 X4]
+0.125000j[X1 Y2]
+0.015625[X1 X2 X3 Y4]
-0.125000j[Y1 X2]
+0.125000[Y1 Y2]
